### Installation

In [38]:
# !pip install folium

### Imports

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import folium
import numpy as np
import pandas as pd
import violencia_obstetrica.data.export as export
import violencia_obstetrica.data.load as load
import violencia_obstetrica.data.process as process
from folium import plugins
from folium.features import CustomIcon, FeatureGroup


### Load data

In [29]:
testimonios_cuba = load.data_testimonios
icon_image = load.icon_image
figures = load.figures

### Read data as dataframe

In [30]:
testimonios = load.load_file(file=testimonios_cuba, sheet="base_limpia")

### Dataframe sample

In [31]:
testimonios.sample(1)

,Testimonio,Provincia,Año,Hospital,ID
149,Entré al hospital el jueves. Tuve al bebé el s...,La Habana,2000.0,Hospital Universitario Ginecobstétrico Eusebio...,584.0


### Clean data

In [32]:
process.clean_dataframe(testimonios, col_name="Hospital")

testimonios.sample(1)

,Testimonio,Provincia,Año,Hospital,ID
168,Recibí la mejor atención médica por parte de m...,La Habana,2020.0,Hospital Universitario Ginecobstétrico Eusebio...,693.0


In [33]:
testimonios["Hospital"] = testimonios["Hospital"].fillna('')

testimonios.loc[testimonios['Hospital'] == '']

,Testimonio,Provincia,Año,Hospital,ID
39,Fui maltratada y sentí falta de empatía. Si mi...,La Habana,1998.0,,109.0
41,Se me explicó la función del monitor y su impo...,La Habana,2017.0,,114.0
48,Te ignoraban cuando solicitabas algo como un r...,Villa Clara,2017.0,,137.0
83,"Dilataron la decisión de hacer cesárea, ponien...",La Habana,1981.0,,292.0
155,Ingresé días antes de dar a luz porque tenía l...,Las Tunas,2012.0,,624.0
182,No me informaron sobre cómo es el trabajo de p...,La Habana,2019.0,,743.0


### Explore data

In [34]:
# Number of unique values from column
print(testimonios['Provincia'].nunique())

# Unique values for a column to a list
provincias = list(testimonios['Provincia'].dropna().unique())
provincias

14


['La Habana',
 'Pinar del Río',
 'Mayabeque',
 'Camagüey ',
 'Isla de la Juventud',
 'Matanzas',
 'Sancti Spíritus',
 'Villa Clara',
 'Ciego de Ávila',
 'Holguín',
 'Artemisa ',
 'Las Tunas',
 'Santiago de Cuba',
 'Cienfuegos']

### Filter data

In [35]:
# Function to filter data according to column values
def filter_prov(df, col="Name", val=list):
    dfs = []
    for i in range(len(provincias)):
        prov = df.loc[df[col] == val[i]]
        dfs.append(prov)
    return list(dfs)

p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14 = filter_prov(testimonios, col="Provincia", val=provincias)

### Create random coordinates for each dataframe

In [36]:
coordinates = [
    [23.03353, 23.09987, -82.15740, -82.50496], # La Habana
    [22.42376, 22.42995, -83.69975, -83.63952], # Pinar del Río
    [22.85195, 22.84857, -81.93923, -81.89057], # Mayabeque
    [21.39445, 21.38787, -77.90326, -77.99985], # Camagüey
    [21.68043, 21.71871, -82.79392, -82.89159], # Isla de la Juventud
    [23.02835, 23.02646, -81.54440, -81.59225], # Matanzas
    [21.94163, 21.93876, -79.43991, -79.44849], # Sancti Spiritus
    [22.56186, 22.45530, -79.94375, -79.98671], # Villa Clara
    [21.84588, 21.84460, -78.75283, -78.75678], # Ciego de Ávila
    [20.88537, 20.88825, -76.25460, -76.28147], # Holguín
    [22.81586, 22.81665, -8275735, -82.75941],  # Artemisa
    [20.96658, 20.96722, -76.95149, -76.95801], # Las Tunas
    [20.02077, 20.02884, -75.81065, -75.82374], # Santiago de Cuba
    [22.16815, 22.15956, -80.42153, -80.42599]  # Cienfuegos
]

"""
Function to create random coordinates
according to dataframes rows length and
within a range of bounding coordinates
for each province in Cuba
"""
def create_coordinates(*args, coor=list):
    dfs = list(args)
    for i in range(len(dfs)):
        x = list(np.random.uniform(coor[i][0], coor[i][1], len(dfs[i])))
        y = list(np.random.uniform(coor[i][2], coor[i][3], len(dfs[i])))
        dfs[i] = dfs[i].assign(Long = x)
        dfs[i] = dfs[i].assign(Lat = y)
    return list(dfs)

p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14 = create_coordinates(p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, coor=coordinates)

### Concatenate dataframes

In [37]:
# Create a list with dataframes
provincias = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14]

# Concatenate the dataframes
df_join = pd.concat(provincias, ignore_index=True)

df_join.sample(1)

,Testimonio,Provincia,Año,Hospital,ID,Long,Lat
49,Durante la gestación tuve varias complicacione...,La Habana,2010.0,Hospital Ginecobstétrico Ramón González Coro,300.0,23.060094,-82.366947


### Create a map

In [39]:
# Create a map at specified location
m = folium.Map(location=[22.037635528256533, -79.36879425670136], zoom_start=7, 
               tiles=None)

# Add layer and set dark as default mode
folium.TileLayer(tiles="Stamen Watercolor", name="Testimonios", attr= '''
               &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> 
               contributors &copy; <a href="https://carto.com/attributions">CARTO</a>
               ''').add_to(m)

# Add fullscreen option
plugins.Fullscreen(
    position="topright",
    title="Pantalla completa",
    title_cancel="Salir de pantalla completa",
    force_separate_button=True,
).add_to(m)

# Add a group
testimonios = FeatureGroup(name="Testimonios")
testimonios.add_to(m)

# Iter a dataframe rows
for i, r in df_join.iterrows():
    # Custom icon image
    icon = CustomIcon('../assets/icon.png', 
    icon_size=(60, 60),
    icon_anchor=(20, 10),
    popup_anchor=(-2, -15))
    # Add markers
    folium.Marker([r['Long'], r['Lat']],
    # Custom popup with html and css style
    popup = folium.Popup('''
                        <div class='text'>
                        <scroll-container>
                            <scroll-page>
                            <p style="text-align:center">{}</p>
                            <hr color=black />
                            <p>{}</p>
                            </scroll-page>
                        </scroll-container>
                        </div>

                        <style type='text/css'>
                            .text {{
                                font-family: Verdana, sans-serif;
                                text-align:justify;
                                font-size:12px;
                                font-weight: 600; 

                                border: black solid 2px;
                                # border-radius: 5px;
                                padding: 20px;
                                box-shadow: 0 -5px 5px -3px black, 0 5px 5px -3px black;
                            }}

                            .leaflet-popup-content-wrapper, .leaflet-popup-tip {{
                                background-image: radial-gradient( circle farthest-corner at 52.1% -29.6%,  rgba(144,17,105,1) 0%, rgba(51,0,131,1) 100.2% );
                            }}

                            .leaflet-popup-content {{
                                background-color: #85FFBD;
                                background-image: linear-gradient(45deg, #85FFBD 0%, #FFFB7D 100%);
                                color:black;
                                }}

                            .leaflet-popup-content:hover {{
                                background-image: radial-gradient( circle farthest-corner at 7.5% 24%,  rgba(237,161,193,1) 0%, rgba(250,178,172,1) 25.5%, rgba(190,228,210,1) 62.3%, rgba(215,248,247,1) 93.8% );
                                color:black;
                                }}
                            scroll-container {{
                                display: block;
                                width: 500px;
                                height: 250px;
                                overflow-y: auto;
                                scroll-behavior: smooth;
                                padding: 20px;
                                }}
                        </style>

                         '''.format(r["Hospital"], r['Testimonio'])),
    tooltip = r['Provincia'],
    icon = icon).add_to(testimonios)

# Save map as 'html'
# export.save_map(m, folder=figures, file="mapa_testimonios")

m